##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guia completo sobre clustering de peso

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Bem-vindo ao guia completo sobre *clustering de peso*, parte do Kit de ferramentas para otimização de modelos do TensorFlow.

Esta página documenta vários casos de uso e mostra como usar a API para cada um. Depois de saber quais APIs são necessárias, encontre os parâmetros e os detalhes de nível inferior na [documentação da API](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/clustering):

- Se quiser ver os benefícios do clustering de peso e saber o que é compatível, confira a [visão geral](https://www.tensorflow.org/model_optimization/guide/clustering).
- Para um único exemplo completo, veja o [exemplo de clustering de peso](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example).

Neste guia, são abordados os seguintes casos de uso:

- Defina um modelo agrupado.
- Faça o checkpoint e desserialize um modelo agrupado.
- Melhore a exatidão do modelo agrupado.
- Somente para implantação, você precisa seguir as etapas para ver os benefícios da compressão.


## Configuração


In [ ]:
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot

input_dim = 20
output_dim = 20
x_train = np.random.randn(1, input_dim).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=output_dim)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(input_dim, input_shape=[input_dim]),
      tf.keras.layers.Flatten()
  ])
  return model

def train_model(model):
  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )
  model.summary()
  model.fit(x_train, y_train)
  return model

def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

setup_model()
pretrained_weights = setup_pretrained_weights()

## Defina um modelo agrupado


### Agrupe um modelo inteiro (sequencial e funcional)

**Dicas** para melhor exatidão do modelo:

- Você precisa passar um modelo pré-treinado com exatidão aceitável para essa API. Treinar modelos do zero com clustering resulta em exatidão inferior.
- Em alguns casos, o clustering de algumas camadas tem um efeito prejudicial na exatidão do modelo. Confira "Agrupe algumas camadas" para ver como pular o clustering das camadas que afetam mais a exatidão.

Para agrupar todas as camadas, aplique `tfmot.clustering.keras.cluster_weights` ao modelo.


In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 3,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

model = setup_model()
model.load_weights(pretrained_weights)

clustered_model = cluster_weights(model, **clustering_params)

clustered_model.summary()

### Agrupe algumas camadas (modelos sequenciais e funcionais)


**Dicas** para melhor exatidão do modelo:

- Você precisa passar um modelo pré-treinado com exatidão aceitável para essa API. Treinar modelos do zero com clustering resulta em exatidão inferior.
- Agrupe camadas finais com parâmetros mais redundantes (por exemplo, `tf.keras.layers.Dense`, `tf.keras.layers.Conv2D`), em vez das camadas iniciais.
- Congele camadas iniciais antes das camadas agrupadas durante o ajuste fino. Trate os números das camadas congeladas como um hiperparâmetro. Empiricamente, o congelamento das camadas mais iniciais é ideal para a API de clustering atual.
- Evite o clustering de camadas críticas (por exemplo, mecanismo de atenção).

**Mais**: a documentação da API `tfmot.clustering.keras.cluster_weights` fornece detalhes sobre como variar a configuração do clustering por camada.

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights)

# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_clustering_to_dense,
)

clustered_model.summary()

### Agrupe camadas convolucionais por canal

O modelo agrupado pode ser passado para mais otimizações, como a [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization). Se a quantização for feita por canal, o modelo também deverá ser agrupado por canal. Isso aumenta a exatidão do modelo agrupado e quantizado.

**Observação:** somente camadas Conv2D são agrupadas por canal

Para agrupar por canal, o parâmetro `cluster_per_channel` deve ser definido como `True`. Ele pode ser definido para algumas camadas ou para todo o modelo.

**Dicas:**

- Se um modelo terá quantização adicional, você pode considerar usar a [técnica de treinamento consciente de quantização (QAT) que preserva os clusters](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

- Pode ser realizado o pruning do modelo antes de aplicar o clustering por canal. Com o parâmetro `preserve_sparsity` definido como `True`, a esparsidade é preservada durante o clustering por canal. Observe que a [técnica de QAT que preserva os clusters e a esparsidade](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization) deve ser usada nesse caso.

### Agrupe a camada do Keras personalizada ou especifique os pesos da camada que serão agrupados

`tfmot.clustering.keras.ClusterableLayer` atende dois casos de uso:

1. Agrupe qualquer camada sem suporte nativo, incluindo uma camada do Keras personalizada.
2. Especifique os pesos de uma camada compatível que serão agrupados.

Por exemplo, o padrão da API é apenas agrupar o kernel da camada `Dense`. O exemplo abaixo mostra como modificá-la para também agrupar o bias. Observe que, ao derivar da camada do Keras, você precisa sobrepor a função `get_clusterable_weights`, onde especifica o nome da variável treinável que será agrupada e a própria variável treinável. Por exemplo, se você retornar uma lista vazia [], então nenhum peso poderá ser agrupado.

**Erro comum:** o clustering do bias geralmente prejudica muito a exatidão do modelo.

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.clustering.keras.ClusterableLayer):

  def get_clusterable_weights(self):
   # Cluster kernel and bias. This is just an example, clustering
   # bias usually hurts model accuracy.
   return [('kernel', self.kernel), ('bias', self.bias)]

# Use `cluster_weights` to make the `MyDenseLayer` layer train with clustering as usual.
model_for_clustering = tf.keras.Sequential([
  tfmot.clustering.keras.cluster_weights(MyDenseLayer(20, input_shape=[input_dim]), **clustering_params),
  tf.keras.layers.Flatten()
])

model_for_clustering.summary()

Você também pode usar `tfmot.clustering.keras.ClusterableLayer` para o clustering de uma camada personalizada do Keras. Para isso, estenda a `tf.keras.Layer` normalmente e implemente as funções `__init__`, `call` e `build`, mas você também precisa estender a classe `clusterable_layer.ClusterableLayer` e implementar:

1. `get_clusterable_weights`, onde você especifica o kernel dos pesos que serão agrupados, como mostrado acima.
2. `get_clusterable_algorithm`, onde você especifica o algoritmo de clustering para o tensor dos pesos. Isso se deve à necessidade de especificar o formato dos pesos da camada personalizada para o clustering. A classe do algoritmo de clustering retornada deve ser derivada da classe `clustering_algorithm.ClusteringAlgorithm`, e a função `get_pulling_indices` deve ser sobreposta. Um exemplo dessa função, que aceita pesos de postos 1D, 2D e 3D, pode ser encontrado [aqui](https://github.com/tensorflow/model-optimization/blob/18e87d262e536c9a742aef700880e71b47a7f768/tensorflow_model_optimization/python/core/clustering/keras/clustering_algorithm.py#L62).

Um exemplo desse caso de uso pode ser encontrado [aqui](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/clustering/keras/mnist_clusterable_layer_test.py).

## Faça o checkpoint e desserialize um modelo agrupado

**Seu caso de uso:** este código só é necessário para o formato de modelo HDF5 (e não pesos HDF5 ou outros formatos).

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights)
clustered_model = cluster_weights(base_model, **clustering_params)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
clustered_model.save(keras_model_file, include_optimizer=True)

# `cluster_scope` is needed for deserializing HDF5 models.
with tfmot.clustering.keras.cluster_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## Melhore a exatidão do modelo agrupado

Para seu caso de uso específico, considere estas dicas:

- A inicialização centroide tem um papel fundamental na exatidão final do modelo otimizado. Em geral, a inicialização kmeans++ supera a inicialização aleatória, de densidade e linear. Quando kmeans++ não está em uso, a inicialização linear tende a superar a inicialização aleatória e de densidade, já que não costuma perder pesos grandes. No entanto, observou-se que a inicialização de densidade proporciona melhor exatidão para o caso de usar alguns clusters em pesos com distribuições bimodais.

- Defina uma taxa de aprendizado mais baixa do que a usada no treinamento ao ajustar o modelo agrupado.

- Para ideias gerais sobre como melhorar a exatidão do modelo, procure dicas para seu caso de uso em "Defina um modelo agrupado".

## Implantação

### Exporte o modelo com a compressão do tamanho

**Erro comum**: o `strip_clustering` e a aplicação de um algoritmo de compressão padrão (por exemplo, por gzip) são necessários para ver os benefícios de compressão do clustering.

In [ ]:
model = setup_model()
clustered_model = cluster_weights(model, **clustering_params)

clustered_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

clustered_model.fit(
    x_train,
    y_train
)

final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print("final model")
final_model.summary()

print("\n")
print("Size of gzipped clustered model without stripping: %.2f bytes" 
      % (get_gzipped_model_size(clustered_model)))
print("Size of gzipped clustered model with stripping: %.2f bytes" 
      % (get_gzipped_model_size(final_model)))